In [12]:
import numpy as np # library for vectorized computation
import pandas as pd # library to process data as dataframes

In [90]:
# read the Wikipedia table
data = pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
df=data[0]

# set column names
df.columns=['PostalCode','Borough','Neighbourhood']


In [91]:
# Remove rows in Borough where value is Not assigned
df = df[df.Borough != 'Not assigned']


In [92]:
# Group any postcode duplicates in rows together
df=df.groupby("PostalCode").agg(lambda x:','.join(set(x)))


In [93]:
# shape of the dataframe
df.shape

(103, 2)

In [94]:
!wget -q -O 'Geospatial_data.csv' http://cocl.us/Geospatial_data

print('Success!')

Success!


In [95]:
geo_df = pd.read_csv('Geospatial_data.csv')


In [96]:
geo_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [97]:
df['Latitude']=geo_df['Latitude'].values
df['Longitude']=geo_df['Longitude'].values

In [98]:
df.head()

,Borough,Neighbourhood,Latitude,Longitude
PostalCode,,,,
M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
M1G,Scarborough,Woburn,43.770992,-79.216917
M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [75]:
!conda install -c conda-forge folium=0.5.0 --yes
import folium

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-4.1.0               |             py_1         614 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    branca-0.4.1               |             py_0          26 KB  conda-forge
    ca-certificates-2020.6.20  |       hecda079_0         145 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    certifi-2020.6.20          |   py36h9f0ad1d_0         151 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    ------------------------------------------------------------
                       

In [102]:
# create map of Toronto using latitude and longitude values

# define centre and produce map
center_lat=df.Latitude.mean()
center_long=df.Longitude.mean()
map_toronto = folium.Map(location=[center_lat, center_long], zoom_start=13)

# add markers to each Neighbourhood
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  

# show map
map_toronto